In [15]:
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [28]:
resource_name = "rg_project_ml"
workspace_name = "w_project_ml"
subscriptionID = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 
aml_compute_target = "projectml" 
experiment_name= 'credit_card_project1'

In [6]:
# Now create Workspace
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')

Workspace is already exist


In [12]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D4_v3",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=10000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

This Compute Target already exist.
Azure Machine Learning Compute attached now


In [29]:
# Create Experiment
exp = Experiment(ws,experiment_name)

In [30]:
# Create environment to execute your code
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="credit_card_detect.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

Submitting c:\Users\User\Untitled Folder 1 directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


RunId: credit_card_project1_1688443712_afd824f1
Web View: https://ml.azure.com/runs/credit_card_project1_1688443712_afd824f1?wsid=/subscriptions/f2813f24-bf98-421d-b7cb-554e68274436/resourcegroups/rg_project_ml/workspaces/w_project_ml&tid=5585330f-9739-4fc3-9558-6526f2590536

Streaming user_logs/std_log.txt

Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 5.702739953994751 seconds

Execution Summary
RunId: credit_card_project1_1688443712_afd824f1
Web View: https://ml.azure.com/runs/credit_card_project1_1688443712_afd824f1?wsid=/subscriptions/f2813f24-bf98-421d-b7cb-554e68274436/resourcegroups/rg_project_ml/workspaces/w_project_ml&tid=5585330f-9739-4fc3-9558-6526f2590536



{'runId': 'credit_card_project1_1688443712_afd824f1',
 'target': 'projectml',
 'status': 'Completed',
 'startTimeUtc': '2023-07-04T04:09:40.894265Z',
 'endTimeUtc': '2023-07-04T04:15:33.922972Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '2c6a0ed6-3cdb-44ca-9358-b83762f91be6',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'credit_card_detect.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'projectml',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': '

In [31]:
#Register model in workspace
from azureml.core import Model
model = Model.register(ws,model_path="./credit_card.pkl",model_name="Credit_Fraud_Model")

Registering model Credit_Fraud_Model


In [33]:
model = Model(ws,"Credit_Fraud_Model")

In [34]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

myenv=Environment(name="credit_fraud-env")
conda_packages = ['numpy','pandas']
pip_packages = ['azureml-sdk','azureml-defaults','scikit-learn']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages, python_version='3.6.2')
myenv.python.conda_dependencies=mycondaenv
myenv.register(workspace=ws)



{
    "assetId": "azureml://locations/eastus/workspaces/af792426-5df2-445b-9938-cdccb6b0bc5f/environments/credit_fraud-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230620.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "credit_fraud-env",
  

In [35]:
inference_config = InferenceConfig(entry_script='model_score.py',source_directory='.',environment=myenv)

In [36]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [ ]:
from azureml.core.model import Model
# Deploying the model
service = Model.deploy(ws,"creditcarddetection", #This is endpoint name
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

###### the API is generated after Deploying the model https://creditcardfraud.eastus.inference.ml.azure.com/score